In [48]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [49]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [50]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@45240991

In [51]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Python/"
val prefix = "delta"

val nohup = spark.read.textFile(s"${folder}${prefix}*")


org.apache.spark.sql.SparkSession$implicits$@46e2b92a

In [52]:
println(nohup.count)

10280


null

In [53]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double)
case class Stage(runID: Long, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)
case class MDFrow(mdfID: Long, n: Long, method: String, epsilon: Double, mu: Int, delta: Int, stage: String, time: Double, load: Int, unit: String)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class Stage
defined class MDFrow
defined class DatasetOps


In [54]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.show(10)

+--------------------+---+
|                line|  n|
+--------------------+---+
|FLOCKFINDER=Spati...|  0|
|WARNING:root:Sett...|  1|
|acald013@dblab-ra...|  2|
|acald013@dblab-ra...|  3|
|acald013@dblab-ra...|  4|
|acald013@dblab-ra...|  5|
|no org.apache.spa...|  6|
|starting org.apac...|  7|
|acald013@dblab-ra...|  8|
|acald013@dblab-ra...|  9|
+--------------------+---+
only showing top 10 rows



null

In [55]:
val indicesRun = lines.filter{ l => 
        l.line.contains("=== MergeLast Start ===") || l.line.contains("method=MergeLast,") ||
        l.line.contains("=== SpatialJoin Start ===") || l.line.contains("method=SpatialJoin,")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.show(10, truncate = false)

+-----+---+
|runID|n  |
+-----+---+
|0    |18 |
|0    |19 |
|0    |20 |
|0    |21 |
|0    |22 |
|0    |23 |
|0    |24 |
|0    |25 |
|0    |26 |
|0    |27 |
+-----+---+
only showing top 10 rows



null

In [56]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr1  = line.split(" -> ")
        val date  = arr1(0)
        val arr2  = arr1(1).split(",")
        val method  = arr2(0).split("=")(1)
        val cores   = arr2(1).split("=")(1).toInt
        val epsilon = arr2(2).split("=")(1).toDouble
        val mu      = arr2(3).split("=")(1).toInt
        val delta   = arr2(4).split("=")(1).toInt
        val time    = arr2(5).split("=")(1).toDouble
        Run(runID, date, method, cores, epsilon, mu, delta, time)
    }.
    cache
runs.show(10, truncate = false)

+-----+-----------------------+-----------+-----+-------+---+-----+----------+
|runID|date                   |method     |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+-----------+-----+-------+---+-----+----------+
|0    |2018-05-29 08:32:13,609|SpatialJoin|28   |30.0   |4  |3    |641.899   |
|1    |2018-05-29 08:42:24,770|MergeLast  |28   |30.0   |4  |3    |590.156   |
|2    |2018-05-29 08:53:22,086|SpatialJoin|28   |30.0   |4  |4    |634.567   |
|3    |2018-05-29 09:03:07,625|MergeLast  |28   |30.0   |4  |4    |560.882   |
|4    |2018-05-29 09:14:10,225|SpatialJoin|28   |30.0   |4  |5    |639.184   |
|5    |2018-05-29 09:23:27,070|MergeLast  |28   |30.0   |4  |5    |533.171   |
|6    |2018-05-29 09:34:41,038|SpatialJoin|28   |30.0   |4  |6    |651.047   |
|7    |2018-05-29 09:43:13,091|MergeLast  |28   |30.0   |4  |6    |489.391   |
|8    |2018-05-29 09:53:47,335|SpatialJoin|28   |30.0   |4  |3    |611.923   |
|9    |2018-05-29 10:08:26,373|MergeLast  |28   |30.

null

In [57]:
val stages = lines.filter(_.line.contains("|")).
    join(indexRun, "n").
    map{ m =>
        val n         = m.getLong(0)
        val line      = m.getString(1)
        val runID     = m.getInt(2)
        var arr1      = line.split(" -> ")
        val timestamp = arr1(0).trim
        val arr2      = arr1(1).split("\\|")
        val stage     = arr2(0).trim
        val time      = arr2(1).trim.dropRight(1).toDouble
        val arr3      = arr2(2).trim.split(" ")
        val load      = arr3(0).toInt
        val unit      = arr3(1)
        Stage(runID, n, timestamp, stage, time, load, unit)
    }.cache
    
stages.count()
stages.show(10, truncate = false)

+-----+---+-----------------------+-------------------------+---------+-----+------------+
|runID|n  |timestamp              |stage                    |stageTime|load |unit        |
+-----+---+-----------------------+-------------------------+---------+-----+------------+
|0    |19 |2018-05-29 08:21:37,271|Reporting locations...   |5.31     |18093|points      |
|0    |35 |2018-05-29 08:22:11,945|1.Set of disks for t_i...|34.67    |205  |disks       |
|0    |36 |2018-05-29 08:22:12,381|4.Found flocks...        |0.44     |0    |flocks      |
|0    |37 |2018-05-29 08:22:13,161|5.Updating times...      |0.48     |205  |flocks      |
|0    |38 |2018-05-29 08:22:14,070|6.Filter phase...        |0.91     |205  |flocks      |
|0    |39 |2018-05-29 08:22:18,768|Reporting locations...   |4.7      |18245|points      |
|0    |55 |2018-05-29 08:22:41,868|1.Set of disks for t_i...|23.1     |216  |disks       |
|0    |56 |2018-05-29 08:22:45,441|2.Distance Join phase... |3.57     |645  |combinations|

null

In [58]:
val ml_stages = stages.join(runs, "runID").
    filter($"method" === "MergeLast").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "Reporting locations at t=\\d+", "0.Reporting locations")).
    withColumn("stage2", regexp_replace($"stage1", "Checking internal timestamps", "4.Checking internals")).
    withColumn("stage3", regexp_replace($"stage2", "\\.\\.\\.", "")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage3".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
ml_stages.show(truncate = false)


+-----+---+---------+-------+---+-----+----------------------------+---------+
|runID|n  |method   |epsilon|mu |delta|stage                       |stageTime|
+-----+---+---------+-------+---+-----+----------------------------+---------+
|1    |291|MergeLast|30.0   |4  |3    |0.Reporting locations       |5.63     |
|1    |307|MergeLast|30.0   |4  |3    |1.Set of disks for t_i      |31.7     |
|1    |308|MergeLast|30.0   |4  |3    |0.Reporting locations       |4.93     |
|1    |324|MergeLast|30.0   |4  |3    |2.Set of disks for t_i+delta|21.81    |
|1    |325|MergeLast|30.0   |4  |3    |3.Joining timestams         |8.46     |
|1    |326|MergeLast|30.0   |4  |3    |4.Checking internals        |12.11    |
|1    |327|MergeLast|30.0   |4  |3    |0.Reporting locations       |4.85     |
|1    |343|MergeLast|30.0   |4  |3    |1.Set of disks for t_i      |19.16    |
|1    |344|MergeLast|30.0   |4  |3    |0.Reporting locations       |4.79     |
|1    |360|MergeLast|30.0   |4  |3    |2.Set of disk

null

In [59]:
val sj_stages = stages.join(runs, "runID").
    filter($"method" === "SpatialJoin").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "\\.\\.\\.", "")).
    withColumn("stage2", regexp_replace($"stage1", "Reporting", "0.Reporting")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage2".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
sj_stages.show(truncate = false)



+-----+---+-----------+-------+---+-----+----------------------+---------+
|runID|n  |method     |epsilon|mu |delta|stage                 |stageTime|
+-----+---+-----------+-------+---+-----+----------------------+---------+
|0    |19 |SpatialJoin|30.0   |4  |3    |0.Reporting locations |5.31     |
|0    |35 |SpatialJoin|30.0   |4  |3    |1.Set of disks for t_i|34.67    |
|0    |36 |SpatialJoin|30.0   |4  |3    |4.Found flocks        |0.44     |
|0    |37 |SpatialJoin|30.0   |4  |3    |5.Updating times      |0.48     |
|0    |38 |SpatialJoin|30.0   |4  |3    |6.Filter phase        |0.91     |
|0    |39 |SpatialJoin|30.0   |4  |3    |0.Reporting locations |4.7      |
|0    |55 |SpatialJoin|30.0   |4  |3    |1.Set of disks for t_i|23.1     |
|0    |56 |SpatialJoin|30.0   |4  |3    |2.Distance Join phase |3.57     |
|0    |57 |SpatialJoin|30.0   |4  |3    |3.Getting candidates  |3.38     |
|0    |58 |SpatialJoin|30.0   |4  |3    |4.Found flocks        |0.76     |
|0    |59 |SpatialJoin|30

null

In [60]:
val data = runs.select($"runID", $"method", $"epsilon", $"mu", $"delta", $"methodTime".alias("time")).
    orderBy($"runID", $"epsilon", $"method").
    cache
data.count()
data.show(10, truncate = false)

+-----+-----------+-------+---+-----+-------+
|runID|method     |epsilon|mu |delta|time   |
+-----+-----------+-------+---+-----+-------+
|0    |SpatialJoin|30.0   |4  |3    |641.899|
|1    |MergeLast  |30.0   |4  |3    |590.156|
|2    |SpatialJoin|30.0   |4  |4    |634.567|
|3    |MergeLast  |30.0   |4  |4    |560.882|
|4    |SpatialJoin|30.0   |4  |5    |639.184|
|5    |MergeLast  |30.0   |4  |5    |533.171|
|6    |SpatialJoin|30.0   |4  |6    |651.047|
|7    |MergeLast  |30.0   |4  |6    |489.391|
|8    |SpatialJoin|30.0   |4  |3    |611.923|
|9    |MergeLast  |30.0   |4  |3    |849.025|
+-----+-----------+-------+---+-----+-------+
only showing top 10 rows



null

In [61]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}methods${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [62]:
val d = ml_stages.union(sj_stages).collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}stages${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [63]:
val indicesMdf = lines.filter{ l => 
        l.line.contains(" -> Setting mu=") || l.line.contains(" ->   berlin0-10,") 
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexMdf = spark.createDataset(indicesMdf)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("mdfID","n")
    .cache
indexMdf.show(15, truncate = false)

+-----+---+
|mdfID|n  |
+-----+---+
|0    |20 |
|0    |21 |
|0    |22 |
|0    |23 |
|0    |24 |
|0    |25 |
|0    |26 |
|0    |27 |
|0    |28 |
|0    |29 |
|0    |30 |
|0    |31 |
|0    |32 |
|0    |33 |
|1    |40 |
+-----+---+
only showing top 15 rows



null

In [64]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("m")).orderBy($"m")
mdfInfo.show(10, truncate = false)

+-----+---+
|mdfID|m  |
+-----+---+
|0    |33 |
|1    |53 |
|2    |75 |
|3    |97 |
|4    |119|
|5    |141|
|6    |163|
|7    |185|
|8    |207|
|9    |229|
+-----+---+
only showing top 10 rows



null

In [65]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("n")).orderBy($"n").
    join(indexRun, "n").
    join(runs, "runID").
    join(lines, "n").
    select($"method", $"epsilon", $"mu", $"delta", $"line").
    withColumn("timestamp", substring($"line", 122, 124)).
    select($"method", $"epsilon", $"mu", $"delta", $"timestamp")

mdfInfo.show(truncate = false)

+-----------+-------+---+-----+---------+
|method     |epsilon|mu |delta|timestamp|
+-----------+-------+---+-----+---------+
|SpatialJoin|30.0   |4  |3    |  0      |
|SpatialJoin|30.0   |4  |3    |  1      |
|SpatialJoin|30.0   |4  |3    |  2      |
|SpatialJoin|30.0   |4  |3    |  3      |
|SpatialJoin|30.0   |4  |3    |  4      |
|SpatialJoin|30.0   |4  |3    |  5      |
|SpatialJoin|30.0   |4  |3    |  6      |
|SpatialJoin|30.0   |4  |3    |  7      |
|SpatialJoin|30.0   |4  |3    |  8      |
|SpatialJoin|30.0   |4  |3    |  9      |
|SpatialJoin|30.0   |4  |3    | 10      |
|MergeLast  |30.0   |4  |3    |  0      |
|MergeLast  |30.0   |4  |3    |  2      |
|MergeLast  |30.0   |4  |3    |  1      |
|MergeLast  |30.0   |4  |3    |  3      |
|MergeLast  |30.0   |4  |3    |  4      |
|MergeLast  |30.0   |4  |3    |  5      |
|MergeLast  |30.0   |4  |3    |  6      |
|MergeLast  |30.0   |4  |3    |  7      |
|MergeLast  |30.0   |4  |3    |  8      |
+-----------+-------+---+-----+---

null

In [66]:
val d = mdfInfo.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}mdfInfo${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null